####  Generate PO - DC mapping list

In [91]:
import os, re, json


path_to_dc = './fatima_group_invoices/updated_data/DC_extracted_txt/' 

po_numbers = os.listdir('./fatima_group_invoices/updated_data/PO/')
po_numbers = [po[3:-4] for po in po_numbers]
mappings = {}
for file in os.listdir(path_to_dc):
    text = open(path_to_dc +  file).read()
    for po in po_numbers:
        if po in text:
            if po not in mappings.keys():
                mappings[po] = []
            mappings[po].append(file)
mappings

{'30407': ['Delivery Challan SO # 302 ffc.xlsx.txt'],
 '31526': ['Fatima Fertilizer RYK DC 1608.txt', 'Fatima DC # 1659.txt'],
 '31999': ['DEST-STINV-PRA0146_recognized.txt'],
 '32090': ['Delivery Challans PO # LPO 32090 and LPO 34352_recognized.txt'],
 '32436': ['Delivery Chalan- LPO- 32436.txt'],
 '33436': ['20190430161432384_recognized.txt'],
 '34352': ['Delivery Challans PO # LPO 32090 and LPO 34352_recognized.txt'],
 '34820': ['New Microsoft Office Excel Worksheet.txt'],
 '34887': ['Delivery Challan No. 8457 (08-07-19).txt'],
 '35045': ['207498 FFCL.txt'],
 '35159': ['002.txt', '003.txt'],
 '35617': ['DC 19710  35617 (FFCL- LOCK PAD).txt'],
 '35712': ['DC 19710  35617 (FFCL- LOCK PAD).txt']}

### Delivery Challan Parsing

In [84]:
import pdfquery
import tabula
import json, os, re,ast
import pandas as pd, numpy as np
import warnings
from fuzzywuzzy import process, fuzz

warnings.filterwarnings('ignore')

In [85]:
# LOAD Delivery Challans
delivery_challans = {}
for filename in os.listdir(path_to_dc):
    if filename.endswith(".txt"):
        text = open(path_to_dc + filename,'r',encoding='utf-8').readlines()
        text = [tl.lower().strip() for tl in text if tl.strip()]
        delivery_challans[filename] = text
        
# LOAD Categories:Attributes 'categories.csv'
categories = pd.read_csv('./invoice_matching/data/item_categories_and_attributes.csv')

# LOAD Keywords for DC mapping
with open('./invoice_matching/data/keywords.json', 'r') as file:
    keywords = json.load(file)

# LOAD purchase orders
po_list = json.load(open('./fatima_group_invoices/updated_data/updated_purchase_orders.json', 'r'))

In [86]:
def get_coordinates_for_description(page, continuation_flag):
    height = float(page.attrib['y1'])
    left = float(page.attrib['x0'])
    right = float(page.attrib['x1'])
    bottom = None
    top = None
    for x in page.iter():
        if not x.text:
            continue
        if x.text.lower().strip() == 'description':
            top = height - float(x.attrib['y1'])
        if 'Total Firm & Fixed'.lower() in x.text.lower():
            bottom = height - float(x.attrib['y0'])
            continuation_flag = False
    if not top:
        top = 5
    if not bottom:
        bottom = height - float(page.attrib['y0'])
    return top, left, bottom, right, continuation_flag


def get_text_of_p_o_no(page):
    height = float(page.attrib['y1'])
    for x in page.iter():
        if not x.text:
            continue
        if 'p.o. number' in x.text.lower().strip():
            top = height - float(x.attrib['y1'])
            bottom = top + 10
            left =  float(x.attrib['x0'])
            return x.text


# header should be same, I'm not checking in this function !
def merge_dataframes(list_of_df):
    column_header = ["Sr #","PO\rLine","Item Code","Description","UOM","Quantity","Unit Price","Amount"]
    final_df = pd.DataFrame()
    for df in list_of_df:
        if type(df) is not pd.DataFrame:
            continue
        if column_header in  df.iloc[0,:].values:
            df = df.iloc[1:,]
        final_df = pd.concat([final_df, df], ignore_index=True, sort=False)
    return final_df


def description_df_to_json(df, filename, po_num):
    response = {
        "purchase_order":po_num,
        "product_desc":[]
    }
    for row in df.iterrows():
        try:
            index = int(row[1]['Sr #'])
            response['product_desc'].append({
                'sr_no':index,
                'PO_line':row[1]['PO\rLine'],
                'item_code':row[1]['Item Code'],
                'product_description':re.sub(r"\r",r"\n",row[1]['Description']), 
                'UOM':row[1]['UOM'],
                'quanity':row[1]['Quantity'],
                'rate':row[1]['Unit Price'],
                'amount':row[1]['Amount']
            })
        except:
            if row[1]['Sr #'] !=  row[1]['Sr #']:
                continue

            if "total amount" in row[1]['Sr #']:
                    values = row[1].values
                    values = [v for v in values if type(v) is str]
                    response['total_amount'] = values
    return response


def get_properties(po_item_desc, categories):
    category_text = re.match(r'^(.*?):', po_item_desc)
    if category_text:
        category_text = category_text[0]
    else:
        category_text = po_item_desc
    category_names = categories.iloc[:,0].values
    match = process.extractOne(category_text.lower().strip(), category_names, scorer=fuzz.token_set_ratio)
    index = np.where(category_names == match[0])[0][0]
    properties = categories.iloc[index, 1]
    return match[0], ast.literal_eval(properties)


def properties_found_in_desc(description_line, properties):
    for prop in properties:
        if prop.lower() in description_line.lower():
            return True
    return False


## changes: search prop:value on multi lines 
# if ":" not found , no property key found, then proceed for next 3 lines
def match_po_items_to_dc(po_item_desc, item_code, dc_desc, categories):
    properties = get_properties(po_item_desc, categories)[1]
    po_item_desc = po_item_desc.split('\n')
    
    properties_matched = []
    for index, desc_line in enumerate(po_item_desc):
        if ":" in desc_line:
            property_match = process.extractOne(desc_line, properties, scorer=fuzz.token_set_ratio)
            if property_match[1] > 95:
                prop, value = desc_line.split(':')[0], desc_line.split(':')[1]
                for indx in range(index+1, len(po_item_desc)):
                    if ":" not in po_item_desc[indx] and not properties_found_in_desc(po_item_desc[indx], properties):
                        value = value + ' '+ po_item_desc[indx]
                    elif indx > index+2:
                        break
                    else:
                        break
                property_value_match = fuzz.partial_token_set_ratio(value, dc_desc)
                if property_value_match > 80:
                    properties_matched.append({prop:value})
    if len(properties_matched) >=1:
        return True, properties_matched
    else:
        item_code_score = fuzz.token_set_ratio(item_code.lower(), dc_desc.lower())
        if item_code_score >85:
            return True, properties_matched
    return False, properties_matched



In [89]:
po_list

[{'data': {'product_desc': [{'PO_line': '99-76-1001901-15',
     'UOM': '50\r18-MAY-201910,750.000\r215.000',
     'amount': nan,
     'item_code': 'BELL:\rTYPE/APPLICATION:ELECTRIC\rDOOR\rELECTRICAL RATING:230VAC\rLOUDNESS:80 DB\rTEXT:*\rMFG:\rFANG FONG:313',
     'product_description': 'EACH',
     'quanity': nan,
     'rate': nan,
     'sr_no': 1}],
   'purchase_order': 'LPO- 34820'},
  'page': 'PO-34820.pdf'},
 {'data': {'product_desc': [{'PO_line': '09-65-1001033-01',
     'UOM': '10002-AUG-2019220,000.000\r2,200.000',
     'amount': nan,
     'item_code': 'FILTER,AIR:\r\rTYPE/APPLICATION:COMPRESSOR\rSTYLE:ELEMENT\rTEXT:(ROLLOMATE)\rHS CODE: 8421.99.0080\rMFG:\rCAMERON PROCESS &\rCOMPRESSION SYSTEMS:ZBM-\r16315-25-1',
     'product_description': 'EACH',
     'quanity': nan,
     'rate': nan,
     'sr_no': 2},
    {'PO_line': '09-65-1001033-02',
     'UOM': '3602-AUG-2019486,000.000\r13,500.000',
     'amount': nan,
     'item_code': 'FILTER,AIR:\rTYPE/APPLICATION:SECOND\rSTAGE\rST

### Mapping (DC-PO) , Version 1


In [90]:
mapping_list = []
for purchase_order in po_list:
    po_map = {
        "page": purchase_order['page'], 
        "delivery_challans":[]
    }
    match_found = False
    for filename in os.listdir(path_to_dc):
        if filename.endswith(".txt"):
            text = open(path_to_dc + filename,'r',encoding='utf-8').readlines()
            text = [tl.lower().strip() for tl in text if tl.strip()]
            # move only if PO number found !
            if fuzz.token_set_ratio(po_map['page'][:-4].lower(), ' '.join(text)) > 95:
                print("\n\nMATCHED: ", po_map['page'][:-4].lower(), ' --> ', filename)
                
                po_map['delivery_challans'].append({"challan":filename, "items_matched":[]})
                key_variations = keywords['description_table']['description']
                for key_item in key_variations:
                    match_key = process.extractOne(key_item.lower().strip(), text, scorer=fuzz.token_set_ratio)
                    items_matched = []
                    if match_key[1] > 80:
                        # entered 'Description' Zone 
                        match_index =  text.index(match_key[0])
                        dc_description_lines = ''.join(text[match_index+1:])
                        for item in purchase_order['data']['product_desc']:
                            match_flag, properties_matched = match_po_items_to_dc(item['product_description'], item['item_code'], dc_description_lines,categories)
                            if match_flag:
                                items_matched.append(item)
                           
                    if items_matched:
                        po_map['delivery_challans'][-1]["items_matched"] = items_matched
                        break   # we don't need to iterate over all description variants, after first match
                print("Items Matched: ",len(items_matched) )
    mapping_list.append(po_map)




MATCHED:  po-30407  -->  Delivery Challan SO # 302 ffc.xlsx.txt
Items Matched:  0


MATCHED:  po-34887  -->  Delivery Challan No. 8457 (08-07-19).txt
Items Matched:  0


MATCHED:  po-34352  -->  Delivery Challans PO # LPO 32090 and LPO 34352_recognized.txt
Items Matched:  0


MATCHED:  po-32090  -->  Delivery Challans PO # LPO 32090 and LPO 34352_recognized.txt
Items Matched:  0


MATCHED:  po-31999  -->  DEST-STINV-PRA0146_recognized.txt
Items Matched:  0


In [217]:
### tests script:

dc = delivery_challans['143695.txt']
po_desc = po_list[5]['data']['product_desc'][0]['product_description']
match_po_items_to_dc( po_desc,'99-07-1710015-04', dc, categories)

# sample input variables !

file ='008547250062.INV.PEK.CO.002.20190124.201719.20190124.201800.47653.txt'
dc = delivery_challans[file]
po_items = [itm['product_description'] for itm in po_list[0]['data']['product_desc']]
item_codes = [itm['item_code'] for itm in po_list[0]['data']['product_desc']]

for i in range(len(po_items)):
    match_flag, properties_matched = match_po_to_dc(po_items[i].replace('\n', ' '), item_codes[i] , ' '.join(dc), categories)
    print(match_flag)

True
True
True
True
True
True
True
True
True


### Purchase Orders - Parse to JSON


In [61]:
file_path = './fatima_group_invoices/updated_data/PO/'
purchase_orders = [f for f in os.listdir(file_path) if f.endswith('.pdf')]
column_header = ["Sr #","PO\rLine","Item Code","Description","UOM","Quantity","Unit Price","Amount"]
po_regex = r"(\bFPO-.\d+\b)|(\bLPO-.\d+\b)"
po_list = []
for file in purchase_orders:
    _list = []
    pdf = pdfquery.PDFQuery(file_path+file)
    print("loading file: ", file)
    pdf.load() 
    pages = pdf.tree.findall('LTPage')
    cont_flag = True
    for page in pages:
        po_text = get_text_of_p_o_no(page)
        if po_text:
            match = re.search(po_regex, po_text)
            if match:
                po_num = match.group(0)
        top, left, bottom, right, cont_flag = get_coordinates_for_description(page, cont_flag)
        df = None
        df = tabula.read_pdf(file_path+file, area=(top,left,bottom,right), pages=page.attrib['pageid'],
                             lattice=True, pandas_options={"names":column_header})

#         if type(df) is not pd.DataFrame:
#             print("not FOUND")
#             df = tabula.read_pdf(file_path+file, area=(topf-5,left,bottom+10,right), pages=page.attrib['pageid'],
#                              lattice=True, pandas_options={"names":column_header})
        _list.append(df)
        if not cont_flag:
            break
    po_obj = {
        "page": file,
        "data" : description_df_to_json(merge_dataframes(_list), file, po_num)
        }
    po_list.append(po_obj)#description_df_to_json(merge_dataframes(_list), file))
    print("items extracted: ", len(po_obj['data']['product_desc']))
    
    
with open("./fatima_group_invoices/updated_data/updated_purchase_orders.json","w") as f:
    json.dump(po_list,f)

print("parsed, check variable 'po_list' !")

loading file:  PO-34820.pdf
items extracted:  1
loading file:  PO-35159.pdf
items extracted:  2
loading file:  PO-31526.pdf
items extracted:  7
loading file:  PO-30407.pdf
items extracted:  1
loading file:  PO-34887.pdf
items extracted:  4
loading file:  PO-33436.pdf
items extracted:  1
loading file:  PO-35617.pdf
items extracted:  2
loading file:  PO-34352.pdf
items extracted:  1
loading file:  PO-35688.pdf
items extracted:  1
loading file:  PO-32090.pdf
items extracted:  6
loading file:  PO-33833.pdf
items extracted:  2
loading file:  PO-35045.pdf
items extracted:  3
loading file:  PO-32436.pdf
items extracted:  1
loading file:  PO-34222.pdf
items extracted:  1
loading file:  PO-31999.pdf
items extracted:  1
loading file:  PO-35712.pdf
items extracted:  1
parsed, check variable 'po_list' !


In [65]:
po_list[2]

{'data': {'product_desc': [{'PO_line': '99-82-9999999-01',
    'UOM': '426-NOV-2018809,000.000202,250.000',
    'amount': nan,
    'item_code': 'DIFFERENTIAL PRESSURE GAUGE\rWITH MANIFOLD,\rTAG: DPI-100, DPI-200,DPI-300,\rDIAPHRAGM TYPE DP INDICATING\rGAUGE,\rDP RANGE : 0 ~ 1 BARG,\rDP FULL SCALE: 100 bar (1450 psig),\rOVER PRESSURE: 125 %,\rFLUID: NATURAL GAS,\rDial size: 100 mm,\rTEMPERATURE NORMAL: 70/120\rDEGF,\rPRESSURE MIN/MAX: 500/ 1440\rPSIG,\rBODY MATERIAL: SS 316,\rTEMPERATURE: -20 TO 60 DEGC (-4\rto 140 degF)\rCONNECTION: 1/4" NPT BOTTOM\rCONNECTION\rACCURACY: +/- 0.5 % OF SPAN.\rMAKE: WIKA/NUOVA\rFIMA/ASHCROFT OR EQUIVALENT',
    'product_description': 'EACH',
    'quanity': nan,
    'rate': nan,
    'sr_no': 5},
   {'PO_line': '99-82-9999999-02',
    'UOM': '826-NOV-201892,400.00011,550.000',
    'amount': nan,
    'item_code': 'NEEDLE VALVE FOR PRESSURE\rGAUGE,\rSIZE: 1/2 ,\rCONNECTON: 1/2 " NPT  FEMALE-\r1/2 " NPT   FEMALE,\rMAKE: ANDRESS GREENWOOD/HY-\rLOK/SWAGELOK/HOKE

### Invoice Parsing

In [396]:
import os, json, re
import numpy as np, pandas as pd
from fuzzywuzzy import fuzz, process

In [397]:
# delivery challans

!ls ./invoice_matching/data/

 008547250062.INV.PEK.CO.002.20190124.201719.20190124.201800.47653.txt
 143695.txt
 144391.txt
'DC 1.txt'
'DC 2.txt'
'DC 3.txt'
'DC 4.txt'
'DC 5.txt'
'Fatima 124791 Shp Docs Stamped.txt'
 FPO-30030.txt
'FPO 31506 invoice value $1800 3.txt'
 FPO-32264.txt
 invoices.json
 keywords.json


In [404]:
with open('./invoice_matching/data/invoices.json', 'r') as file:
    invoices = json.load(file)
    
with open('./invoice_matching/data/keywords.json', 'r') as file:
    keywords = json.load(file)


In [ ]:
def map_invoices_for_description(list_of_dc, list_of_invoices):
    # TODO
    return pass

In [402]:
for mapping in invoices_mapping_list:
    print("invoice:", mapping['page'])
    print("DC:  ", [file['challan'] for file in mapping['delivery_challans']])

invoice: 7
DC:   ['Fatima 124791 Shp Docs Stamped.txt']
invoice: 8&9
DC:   ['FPO-32264.txt']
invoice: 10
DC:   []
invoice: 11
DC:   ['008547250062.INV.PEK.CO.002.20190124.201719.20190124.201800.47653.txt']
invoice: 30
DC:   ['143695.txt']
invoice: 31
DC:   ['143695.txt']


In [403]:
inv_desc = invoices[3]['data']['product_desc'][1]['product_description']
filename = '008547250062.INV.PEK.CO.002.20190124.201719.20190124.201800.47653.txt'
dc = open('/home/umair/Downloads/OCRd/OCRd/' + filename,'r',encoding='utf-8').readlines()
invoice_item_matched_to_dc(inv_desc, dc)

True

###  Invoice Parsing